# Monte Carlo: Estimating $\pi$

Notebook structure: 
- What is MC (figure out explanation of why it traverses the probability distribution well)
- Explain how we can use this to find pi (explain that we do a trivial example first) maths exercise
- Now write the naive function to calculate an estimate for `n_samples`
- Now vectorise this function -> show the speed differences (introduction to *magic functions*)
- Show that we can do the same for a sphere section (what are the convergence properties for this?

The essence of Monte Carlo is to take repeated samples (in this case from a probability distribution to obtain numerical results. Often times the randomness of the Monte Carlo method is used for problems that are deterministic in principle. The origins of the name comes from the code name to secret works of von-Neumann and Ulam in the second world war.

![title](figs/pi_estimate.gif)

Our first example is the *Hello World* of Monte Carlo where we will estimate $\pi$ by picking numbers from a 2D uniform distribution. Here the analytic solution is easy to solve so it serves as an intutibe example as we see our estimate converge towards $\pi$. The animation above shows the main idea of uniformally picking points from in a 1x1 grid and then see whether these points would lie in a quarter-circle. From this we can estimate $\pi$.

## The estimator, $\hat \pi$

The equation of a circle is $x^2 + y^2 = 1$ therefore if $x_0^2 + y_0^2 < 1$ the point $(x_0, y_0)$ lies within the circle. Therefore if we have $N_\text{tot}$ samples, we will get a subset $N_\text{circ}$ which lie within the circle. Since the samples are uniform, on expectation the ratio $E[N_\text{circ} / N_\text{tot}] = A_\text{circ} / A_\text{tot}$ where the $A$'s are the areas. 

$$ E\left[\frac{N_\text{circ}}{N_\text{tot}}\right] = \frac{A_\text{circ}}{A_\text{tot}} = \frac{\hat \pi \cdot 1^1 /4}{1^2} = \frac \pi 4 $$

Taking away the expectation and changing the analyic value of $\pi$ to our estimator $\hat \pi$:

$$ \frac{N_\text{circ}}{N_\text{tot}} =  \frac {\hat \pi}{4} $$

$$\hat \pi = 4 \cdot  \frac{N_\text{circ}}{N_\text{tot}}$$

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma

### Exercise: Constructing a naive estimator

The first task is to construct a naive estimator but sequentially picking random points and keeping a running total of the number of these points which lie within the circle then using this to estimate $\pi$. The beauty of a simple problem like this is that we know the value we are aiming for $ \pi = 3.141\cdots$ so you should test our your estimator for different values of `n_samples` n.b. that convergence is quite slow so you need a few hundred samples before you start to get good estimates. 

In [11]:
def estimate_pi_naive(n_samples: int) -> float:
    '''
    YOUR CODE HERE! 

    hint: check the docs for `np.random.uniform()`
    '''

In [12]:
estimate_pi_naive(n_samples=10000)

### Exercise: Constructing a vectorised estimator

Now having constructed an estimator by sequentially picking points we can construct a more efficient estimator. This is done via picking all the points at once and then checking them all at once too. This is done using `numpy`

In [13]:
def estimate_pi_vectorised(n_samples: int) -> float:
    '''
    YOUR CODE HERE!

    hint: use `np.where()`
    '''

In [14]:
estimate_pi_vectorised(n_samples=10000)

Now lets see the how the two different ways of writing this estimate differ in their time performance using the *magic function* `%timeit`. This type of function is only available in Jupyter notebooks.

Our vectorised functions should run about ~10x quicker here as the for loops in the vectorised version are run implicitly in a quicker compiled language, C.

In [16]:
n_samples = int(1e6) # one million samples

%timeit estimate_pi_naive(n_samples)
%timeit estimate_pi_vectorised(n_samples)

2l/pi = n_hit / n_tot

### Extension - Sampling in higher dimensions
As an extension we can further investigate this problem by looking at how the variance of our estimator changes as we perform this experiment in different dimensions. Does generating points in 3D and calculating intersections with a sphere give higher or lower variance? We have quoted some results for volumes of spheres in arbitrary dimension $d$.

$$ V_s/V_t ={\frac {\pi ^{d/2}}{\Gamma {\bigl (}{\tfrac {d}{2}}+1{\bigr )}}}$$

$$ N_s/N_t={\frac {\hat \pi_d ^{d/2}}{\Gamma {\bigl (}{\tfrac {d}{2}}+1{\bigr )}}}$$

$$ \hat \pi_d = \left ( \Gamma(\frac d 2+1) \cdot N_s/N_t \right )^{2/d}$$

$$\mathrm{Var}(\hat \pi_{d,n}) = \frac{\left ( \Gamma(\frac d 2+1) \cdot N_s/N_t \right )^{2/d} - \pi^2}{n}$$

$$\mathrm{Var}(\hat \pi_{2,n}) = E[\hat \pi^2] - E[\hat \pi]^2 = \frac{\left ( 4 \pi - \pi^2 \right )}{n}$$